In [1]:
import numpy as np
import pandas as pd

In [2]:
# get the list of words from Knuth's list of five-letter words
five = pd.read_csv("https://www-cs-faculty.stanford.edu/~knuth/sgb-words.txt",header=None,names=["word"])

In [3]:
five.iloc[400:405] #check that string "false" is not treated like a boolean

,word
400,ended
401,chord
402,false
403,skill
404,holes


In [4]:
def wordle(answer,guess):
    sig=["B","B","B","B","B"]
    for i in range(5):
        if answer[i]==guess[i]:
            sig[i]="G"
        elif guess[i] in answer:
            sig[i]="Y"
    ans=""
    for char in sig:
        ans+=char
    return ans

In [5]:
answer="ready"
guess="rhyme"
wordle(answer,guess)

'GBYBY'

In [6]:
# apply a guess to every word in the list
guess="ready"
five["word"].apply(lambda x: wordle(x,guess)).head(10)

0    BBBBB
1    YYBBB
2    YYBBB
3    BBYBB
4    BBBYB
5    BYBBB
6    YYBBB
7    YBBGB
8    BBBYB
9    YYBBB
Name: word, dtype: object

In [7]:
# given a series of categorical values compute the information entropy
def entropy(series):
    dic={}
    n=len(series)
    for el in series:
        if el not in dic:
            dic[el]=1
        else:
            dic[el]+=1
    e=0
    for key in dic:
        p=dic[key]/n
        if p>0:
            e -= p*np.log(p)
    return e

In [8]:
# compute the entropy of the responses as a probability distribution for a given guess
guess="rates"
entropy(five["word"].apply(lambda x: wordle(x,guess)))

4.239437315510327

In [9]:
# Determine the best next guess by choosing the word which maximizes information entropy
# considers all words as potent
def nextWord(df,allWords):
    ans=""
    m=0
    for word in allWords:
        series=df.word.apply(lambda x: wordle(x,word))
        e=entropy(series)
        if e>m:
            m=e
            ans=word
    return ans

In [10]:
# prompt user to respond with the Wordle response, e.g. as "BYYBG"
def getResponse():
    while(True):
        print("Enter response: ")
        string=input()
        string=string.upper()
        if len(string)==0:
            return ""
        if len(string)!=5:
            print("Response must be exactly 5 characters\n")
        else:
            done=1
            for char in string:
                if char not in ("B","G","Y"):
                    print("Enter Wordle response using 'B','Y','G'")
                    done=0
                    break
            if done==1:
                break
    return string

In [11]:
# Recursively find the best word until solved
def solver(df):
    while(True):
        guess=nextWord(df,five.word)
        print("Next guess:")
        print("\t",guess)
        sig=getResponse()
        if sig=="":
            break
        df=df[df.word.apply(lambda x: wordle(x,guess))==sig]
        if df.shape[0]==1:
            print("Then the word must be:")
            print("\t",df.word.iloc[0])
            break
        if df.shape[0]==0:
            print("No such word was found.")
            break

In [12]:
solver(five)

Next guess:
	 tares
Enter response: 
YBYBB
Next guess:
	 fount
Enter response: 
BGBBG
Then the word must be:
	 robot
